# Focus Buddy - AI Concierge Agent

**Track:** Concierge Agents  
**Model:** Google Gemini 2.0 Flash (Free)  
**Concepts:** Multi-agent systems, Function calling, Session memory

---

## Problem
Knowledge workers struggle to decide what to work on in short time windows, leading to analysis paralysis and poor prioritization.

## Solution
Focus Buddy is a multi-agent system that transforms messy to-do lists into realistic 25-minute focus plans using intelligent prioritization and time-bounded scheduling.

In [ ]:
# Install dependencies
!pip install google-generativeai pydantic -q

In [ ]:
import os
from typing import List, Optional, Dict, Any
from pydantic import BaseModel
import re
import google.generativeai as genai

# Configure API key from Kaggle Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key=api_key)

print("✓ Environment configured")

## Tool Definitions

Three Python functions enable structured workflow:
1. **parse_tasks** - Convert messy text to structured data
2. **prioritize_tasks** - Score and rank by urgency/importance
3. **create_focus_schedule** - Build realistic timelines

In [ ]:
# Data models
class Task(BaseModel):
    """Structured task representation"""
    title: str
    estimated_minutes: int = 10
    deadline: Optional[str] = None
    priority_score: float = 0.0
    completed: bool = False

class ScheduledBlock(BaseModel):
    """A time block in the focus schedule"""
    start_minute: int
    end_minute: int
    task_title: str

# Tool 1: Parse Tasks
def parse_tasks(raw_text: str) -> List[Task]:
    """Parse messy task input into structured Task objects"""
    tasks = []
    lines = re.split(r'[\n\r]+|(?:^|\s)[-•*]\s+|\d+\.\s+', raw_text)
    
    for line in lines:
        line = line.strip()
        if not line or len(line) < 3:
            continue
            
        time_match = re.search(r'\((\d+)\s*min\)|\b(\d+)m\b', line, re.IGNORECASE)
        estimated_minutes = 10
        
        if time_match:
            estimated_minutes = int(time_match.group(1) or time_match.group(2))
            line = re.sub(r'\(?\d+\s*min?\)?', '', line, flags=re.IGNORECASE).strip()
        
        deadline = None
        deadline_match = re.search(r'\b(due|deadline|by)\s+([a-zA-Z]+\s+\d+|\d+/\d+|today|tomorrow)', line, re.IGNORECASE)
        if deadline_match:
            deadline = deadline_match.group(2)
        
        tasks.append(Task(
            title=line,
            estimated_minutes=estimated_minutes,
            deadline=deadline
        ))
    
    return tasks

# Tool 2: Prioritize Tasks
def prioritize_tasks(tasks: List[Task]) -> List[Task]:
    """Sort tasks by priority score"""
    for task in tasks:
        score = 0.0
        
        if task.deadline:
            deadline_lower = task.deadline.lower()
            if 'today' in deadline_lower or 'urgent' in deadline_lower:
                score += 3
            elif 'tomorrow' in deadline_lower:
                score += 2
        
        if task.estimated_minutes <= 10:
            score += 2
        
        title_lower = task.title.lower()
        if any(word in title_lower for word in ['urgent', 'important', 'blocking', 'asap', 'critical']):
            score += 2
        if any(word in title_lower for word in ['bug', 'fix', 'broken', 'error']):
            score += 1.5
        
        score += (60 - min(task.estimated_minutes, 60)) / 100
        task.priority_score = score
    
    return sorted(tasks, key=lambda t: t.priority_score, reverse=True)

# Tool 3: Create Schedule
def create_focus_schedule(tasks: List[Task], available_minutes: int = 25) -> List[ScheduledBlock]:
    """Build a realistic schedule that fits in available_minutes"""
    schedule = []
    current_minute = 0
    buffer_minutes = 3
    usable_minutes = available_minutes - buffer_minutes
    
    for task in tasks:
        if current_minute + task.estimated_minutes > usable_minutes:
            remaining = usable_minutes - current_minute
            if remaining >= 5:
                schedule.append(ScheduledBlock(
                    start_minute=current_minute,
                    end_minute=current_minute + remaining,
                    task_title=f"{task.title} (partial - {remaining} min)"
                ))
                current_minute += remaining
            break
        
        schedule.append(ScheduledBlock(
            start_minute=current_minute,
            end_minute=current_minute + task.estimated_minutes,
            task_title=task.title
        ))
        current_minute += task.estimated_minutes
        
        if len(schedule) >= 4:
            break
    
    if current_minute < available_minutes:
        schedule.append(ScheduledBlock(
            start_minute=current_minute,
            end_minute=available_minutes,
            task_title="Wrap up & notes for next session"
        ))
    
    return schedule

# Function executor
def execute_function(function_name: str, function_args: Dict[str, Any]) -> Any:
    """Execute tool and return results"""
    if function_name == "parse_tasks":
        result = parse_tasks(function_args["raw_text"])
        return [task.dict() for task in result]
    
    elif function_name == "prioritize_tasks":
        tasks = [Task(**t) for t in function_args["tasks"]]
        result = prioritize_tasks(tasks)
        return [task.dict() for task in result]
    
    elif function_name == "create_focus_schedule":
        tasks = [Task(**t) for t in function_args["tasks"]]
        available = function_args.get("available_minutes", 25)
        result = create_focus_schedule(tasks, available)
        return [block.dict() for block in result]
    
    return {"error": f"Unknown function: {function_name}"}

print("✓ Tools defined successfully")

## Agent Configuration

The agent uses a multi-role system:
- **Planner Agent**: Analytical role (parse, prioritize, schedule)
- **Coach Agent**: Interactive role (present, motivate, check-in)

In [ ]:
# System prompt
FOCUS_BUDDY_PROMPT = """
You are Focus Buddy, a concierge AI agent that helps users turn messy to-do lists into realistic 25-minute focus plans.

You operate as a multi-agent system with two roles:
1. Planner Agent: Parse tasks, prioritize by urgency/importance, create time-bounded schedules
2. Coach Agent: Present plans clearly, motivate users, conduct check-ins

Workflow:
1. Ask for time window (default 25 min) and energy level (low/medium/high)
2. Use tools: parse_tasks → prioritize_tasks → create_focus_schedule
3. Present plan with: Summary, Timeline, Checklist, Check-in instructions
4. After session, track completion and suggest improvements

Guidelines:
- Limit to 3-5 tasks per 25 minutes
- Respect user energy (tired = smaller tasks)
- Be concise and practical
- Use bullet lists and mini-timelines
"""

# Tool schemas for Gemini
tools = [{
    "function_declarations": [
        {
            "name": "parse_tasks",
            "description": "Parse messy task text into structured Task objects",
            "parameters": {
                "type": "object",
                "properties": {
                    "raw_text": {"type": "string", "description": "Raw task input"}
                },
                "required": ["raw_text"]
            }
        },
        {
            "name": "prioritize_tasks",
            "description": "Sort tasks by priority score",
            "parameters": {
                "type": "object",
                "properties": {
                    "tasks": {"type": "array", "description": "List of tasks", "items": {"type": "object"}}
                },
                "required": ["tasks"]
            }
        },
        {
            "name": "create_focus_schedule",
            "description": "Create time-bounded schedule",
            "parameters": {
                "type": "object",
                "properties": {
                    "tasks": {"type": "array", "items": {"type": "object"}},
                    "available_minutes": {"type": "integer", "default": 25}
                },
                "required": ["tasks"]
            }
        }
    ]
}]

# Create model
model = genai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    tools=tools,
    system_instruction=FOCUS_BUDDY_PROMPT
)

print("✓ Agent configured")

## Demo Session

Demonstration of the complete workflow with a realistic scenario.

In [ ]:
def run_demo():
    """Run demonstration of Focus Buddy"""
    
    # Sample user input
    user_message = """I have 25 minutes before my next meeting and I'm pretty tired.
    Here's what I need to do:
    
    - Review 3 open pull requests
    - Fix the authentication bug (it's blocking QA)
    - Write API documentation for the /users endpoint (probably 30 min)
    - Reply to Sarah about Q4 planning meeting
    - Prep slides for Friday's demo (need about 10 slides)
    """
    
    print("="*60)
    print("USER INPUT:")
    print("="*60)
    print(user_message)
    print("\n" + "="*60)
    print("AGENT PROCESSING:")
    print("="*60 + "\n")
    
    # Start chat
    chat = model.start_chat()
    response = chat.send_message(user_message)
    
    # Handle function calls
    iteration = 0
    max_iterations = 10
    session_memory = {}
    
    while iteration < max_iterations:
        if not response.candidates:
            break
            
        parts = response.candidates[0].content.parts
        if not parts or not hasattr(parts[0], 'function_call'):
            break
        
        # Process function calls
        function_responses = []
        
        for part in parts:
            if hasattr(part, 'function_call'):
                function_call = part.function_call
                function_name = function_call.name
                function_args = dict(function_call.args)
                
                print(f"🔧 Calling: {function_name}")
                
                # Execute function
                result = execute_function(function_name, function_args)
                
                # Store in memory
                if function_name == "create_focus_schedule":
                    session_memory["current_focus_plan"] = result
                
                # Prepare response
                function_responses.append(
                    genai.protos.Part(
                        function_response=genai.protos.FunctionResponse(
                            name=function_name,
                            response={"result": result}
                        )
                    )
                )
        
        # Send results back
        if function_responses:
            response = chat.send_message(
                genai.protos.Content(parts=function_responses)
            )
        else:
            break
        
        iteration += 1
    
    # Extract final response
    final_text = ""
    if response.candidates and response.candidates[0].content.parts:
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'text'):
                final_text += part.text
    
    print("\n" + "="*60)
    print("FOCUS BUDDY RESPONSE:")
    print("="*60)
    print(final_text)
    print("="*60)
    
    # Show session memory
    if session_memory:
        print("\n📊 SESSION MEMORY:")
        print("-"*60)
        if "current_focus_plan" in session_memory:
            print(f"✓ Focus plan stored with {len(session_memory['current_focus_plan'])} blocks")
        print("-"*60)

# Run the demo
run_demo()

## Concepts Demonstrated

### 1. Multi-Agent System
- **Planner Agent**: Analytical role that parses, prioritizes, and schedules
- **Coach Agent**: Interactive role that presents, motivates, and checks in
- Both roles coordinate through shared system prompt and session memory

### 2. Function Calling (Tools)
- `parse_tasks()`: Converts unstructured text → Task objects
- `prioritize_tasks()`: Applies scoring logic → Sorted list
- `create_focus_schedule()`: Fits tasks into time → Schedule blocks

### 3. Session Memory
- Stores current focus plan across tool calls
- Enables continuity between planning and check-in
- Tracks completion status for multi-sprint sessions

### 4. Evaluation Loop
- Post-session check-ins analyze what worked
- Feedback drives plan adjustments
- Continuous improvement of time estimates

---

## Value Proposition

**Reduces cognitive load**: No more staring at lists wondering what to do  
**Realistic planning**: Fits actual available time instead of wishful thinking  
**Creates momentum**: Small wins in 25 minutes beat perfect plans that never start  
**Repeatable process**: Works for Pomodoro cycles, meeting gaps, or deep work sessions

Ideal for developers, students, writers, and anyone who struggles with task paralysis.

---

## Implementation Notes

- **API**: Google Gemini 2.0 Flash (free tier, 60 requests/min)
- **Architecture**: Single LLM with role-playing (Planner + Coach)
- **Tools**: Three deterministic Python functions
- **Memory**: In-memory dictionary (production would use database)

GitHub Repository: [YOUR_REPO_LINK]